In [13]:
import nltk
from nltk import tokenize
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
# Download the 'punkt' resource
nltk.download()
import os
train_data=pd.read_csv('propaganda_train.tsv',sep='\t')
test_data=pd.read_csv('propaganda_val.tsv',sep='\t')
print(train_data)
print(train_data)
training_df=pd.DataFrame(train_data,columns=["tagged_in_context","label"])
testing_df=pd.DataFrame(test_data,columns=["tagged_in_context","label"])

print(training_df)
labellist=sorted(list(set(training_df['label'].unique()).union(set(testing_df['label'].unique()))))

labels={label:i for i,label in enumerate(labellist)}
labels
reverse_index={value:key for (key,value)in labels.items()}
training_df.head()

AttributeError: module 'nltk' has no attribute 'download'

In [ ]:
# Preprocess the data
training_df['sentences'] = training_df['tagged_in_context'].apply(lambda x: nltk.tokenize.word_tokenize(x.lower()))
training_df['binary_label'] = (training_df['label'] != 'not_propaganda').astype(int)

# Train the Word2Vec model
model = Word2Vec(sentences=train_data['sentences'], vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
# Define a function to convert a sentence to a feature vector
def sentence_to_feature_vector(sentence, word2vec_model):
    feature_vector = np.zeros(word2vec_model.vector_size)
    count = 0
    for word in sentence:
        if word in word2vec_model.wv:
            feature_vector += word2vec_model.wv[word]
            count += 1
    return feature_vector / count if count != 0 else feature_vector

In [ ]:
# Convert the sentences to feature vectors
train_vectors = np.vstack(train_data['sentences'].apply(lambda x: sentence_to_feature_vector(x, model)))
test_vectors = np.vstack(test_data['sentences'].apply(lambda x: sentence_to_feature_vector(x, model)))

# Train the logistic regression classifier
clf = LogisticRegression(random_state=42).fit(train_vectors, train_data['binary_label'])

# Make predictions and evaluate the performance
predictions = clf.predict(test_vectors)
print(classification_report(test_data['binary_label'], predictions))

In [ ]:
import torch
use_cuda=torch.cuda.is_available()
if use_cuda:
  print("GPU acceleration enabled")
else:
  print("GPU acceleration NOT enabled.  If using Colab, have you changed the runtype type and selected GPU as the hardware accelerator?")
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()
# get number of GPUs available
torch.cuda.device_count() # returns 1 in my case


In [14]:
%pip uninstall nltk
%pip install -U nltk

^C
Note: you may need to restart the kernel to use updated packages.
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7
Note: you may need to restart the kernel to use updated packages.
